In [2]:
import numpy as np
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, MultiHeadAttention, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import os

# Number of Folds
NUM_FOLDS = 5
fold_files = [f"train_2d_fold{i}.pkl" for i in range(1, 6)]

# Directory to Save Models & Scaler
os.makedirs("2Ddata_Transformer", exist_ok=True)

# Label Columns
label_columns = ["Positive_Emotions", "Negative_Emotions", "Self_Esteem", "Meaning_in_Life", "Social_Support"]

# ===== Step 1: Compute Global Max Sequence Length Across All Folds ===== #
global_max_seq_length = 0

for file_name in fold_files:
    with open(file_name, "rb") as f:
        fold_dict = pickle.load(f)

    max_length = max(len(seq) for seq in fold_dict["data"])
    global_max_seq_length = max(global_max_seq_length, max_length)

print(f"\n✅ Global Max Sequence Length: {global_max_seq_length}")

# ===== Step 2: Standardize Features Using a Single Scaler ===== #
scaler = StandardScaler()

# Fit scaler on the first fold's training data
with open(fold_files[0], "rb") as f:
    first_fold_dict = pickle.load(f)

columns = first_fold_dict["columns"]
label_indices = [columns.index(col) for col in label_columns]
feature_indices = [i for i in range(len(columns)) if i not in label_indices and columns[i] != "participant_id"]

X_raw_first_fold = [np.array(seq[:, feature_indices], dtype=np.float32) for seq in first_fold_dict["data"]]
X_flattened = np.vstack(X_raw_first_fold)
scaler.fit(X_flattened)

# Save Scaler
with open("2Ddata_Transformer/scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("✅ StandardScaler saved!")

2025-02-28 12:59:58.483393: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-28 12:59:58.522146: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-28 12:59:58.522170: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-28 12:59:58.522191: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-28 12:59:58.529904: I tensorflow/core/platform/cpu_feature_g


✅ Global Max Sequence Length: 74
✅ StandardScaler saved!


In [4]:
# ===== Step 3: Train Transformer Model Across Folds ===== #
cv_results = {"train_loss": [], "val_loss": [], "train_mae": [], "val_mae": [], "residual_error": []}

for fold, file_name in enumerate(fold_files):
    print(f"\n🟢 Processing Fold {fold+1}/{NUM_FOLDS}")

    # Load fold data
    with open(file_name, "rb") as f:
        fold_dict = pickle.load(f)

    # Extract features and labels
    X_raw = [np.array(seq[:, feature_indices], dtype=np.float32) for seq in fold_dict["data"]]
    y_raw = [np.array(seq[:, label_indices], dtype=np.float32) for seq in fold_dict["data"]]

    # Standardize Features using the pre-saved scaler
    X_flat = np.vstack(X_raw)
    X_scaled = scaler.transform(X_flat)

    # Restore sequence structure
    X_fixed = []
    start = 0
    for seq in X_raw:
        length = len(seq)
        X_fixed.append(X_scaled[start:start+length])
        start += length

    # Convert lists to NumPy arrays
    X_fixed = np.array(X_fixed, dtype=object)

    # ✅ Apply the global max sequence length for consistent padding
    X_padded = tf.keras.preprocessing.sequence.pad_sequences(X_fixed, maxlen=global_max_seq_length, dtype="float32", padding="post")

    # Only take the last timestep of labels
    y_final = np.array([seq[-1] for seq in y_raw], dtype=np.float32)

    # ===== Define Transformer Model ===== #
    def build_transformer_model(input_shape):
        inputs = Input(shape=input_shape)

        # Self-Attention Layer
        attn_output = MultiHeadAttention(num_heads=4, key_dim=64)(inputs, inputs)
        attn_output = Dropout(0.3)(attn_output)
        attn_output = LayerNormalization(epsilon=1e-6)(attn_output + inputs)  # Residual Connection

        # Feed-Forward Network
        ffn = Dense(128, activation="relu", kernel_regularizer=l2(0.01))(attn_output)
        ffn = Dropout(0.3)(ffn)
        ffn = Dense(64, activation="relu", kernel_regularizer=l2(0.01))(ffn)

        # Output Layer (Regression)
        outputs = Dense(5, activation="linear")(ffn[:, -1, :])  # Take last timestep output

        model = Model(inputs, outputs)
        model.compile(optimizer=Adam(learning_rate=0.0003), loss="mae", metrics=["mae"])
        return model

    model = build_transformer_model((X_padded.shape[1], X_padded.shape[2]))

    # Train Model
    history = model.fit(X_padded, y_final, epochs=100, batch_size=16, validation_split=0.2, verbose=1)

    # Save Model
    model.save(f"2Ddata_Transformer/transformer_model_fold{fold+1}.keras")

    # Store Results
    train_loss, train_mae = history.history["loss"][-1], history.history["mae"][-1]
    val_loss, val_mae = history.history["val_loss"][-1], history.history["val_mae"][-1]

    # Predict on train set for residual analysis
    y_pred = model.predict(X_padded)
    residuals = y_final - y_pred
    mean_residual = np.mean(residuals)
    std_residual = np.std(residuals)

    # Append results
    cv_results["train_loss"].append(train_loss)
    cv_results["val_loss"].append(val_loss)
    cv_results["train_mae"].append(train_mae)
    cv_results["val_mae"].append(val_mae)
    cv_results["residual_error"].append(mean_residual)

    # Save Metrics
    fold_metrics = {
        "loss": history.history["loss"],
        "val_loss": history.history["val_loss"],
        "mae": history.history["mae"],
        "val_mae": history.history["val_mae"]
    }
    with open(f"2Ddata_Transformer/training_metrics_fold{fold+1}.pkl", "wb") as f:
        pickle.dump(fold_metrics, f)

    # Save Residuals
    residual_data = {"mean_residual": mean_residual, "std_residual": std_residual}
    with open(f"2Ddata_Transformer/residuals_fold{fold+1}.pkl", "wb") as f:
        pickle.dump(residual_data, f)

    print(f"🔹 Fold {fold+1} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Train MAE: {train_mae:.4f}, Val MAE: {val_mae:.4f}, Residual Error: {mean_residual:.4f} ± {std_residual:.4f}")

# ✅ Final Summary
print("\n✅ All Transformer models trained and saved with consistent padding and standardization!")


🟢 Processing Fold 1/5


2025-02-28 13:00:18.591326: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 701 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:3b:00.0, compute capability: 7.5
2025-02-28 13:00:18.593821: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13664 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:d8:00.0, compute capability: 7.5


Epoch 1/100


2025-02-28 13:00:21.854671: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fac691c2310 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-02-28 13:00:21.854710: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-02-28 13:00:21.854721: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
2025-02-28 13:00:21.900532: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-02-28 13:00:21.979380: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2025-02-28 13:00:22.210489: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


11/11 [==============================] - 6s 38ms/step - loss: 6.9667 - mae: 4.6374 - val_loss: 6.0270 - val_mae: 3.7175
Epoch 2/100
11/11 [==============================] - 0s 19ms/step - loss: 6.0414 - mae: 3.7458 - val_loss: 5.4678 - val_mae: 3.1899
Epoch 3/100
11/11 [==============================] - 0s 19ms/step - loss: 5.3984 - mae: 3.1334 - val_loss: 5.0042 - val_mae: 2.7560
Epoch 4/100
11/11 [==============================] - 0s 19ms/step - loss: 4.7728 - mae: 2.5371 - val_loss: 4.5550 - val_mae: 2.3358
Epoch 5/100
11/11 [==============================] - 0s 19ms/step - loss: 4.2978 - mae: 2.0914 - val_loss: 4.1678 - val_mae: 1.9781
Epoch 6/100
11/11 [==============================] - 0s 19ms/step - loss: 3.8976 - mae: 1.7206 - val_loss: 4.1201 - val_mae: 1.9603
Epoch 7/100
11/11 [==============================] - 0s 19ms/step - loss: 3.6184 - mae: 1.4728 - val_loss: 3.9326 - val_mae: 1.8070
Epoch 8/100
11/11 [==============================] - 0s 19ms/step - loss: 3.5605 - mae: 

In [ ]:
print(f"First 5 training samples:\n{X_padded[:5]}\n")
print(f"First 5 validation samples:\n{X_padded[int(0.8 * len(X_padded)):int(0.85 * len(X_padded))]}\n")


In [6]:
import numpy as np
import pickle
import tensorflow as tf
import os
from sklearn.preprocessing import StandardScaler

# Number of Folds
NUM_FOLDS = 5
fold_files = [f"train_2d_fold{i}.pkl" for i in range(1, 6)]
test_file = "test_2d_fold5.pkl"  # Ensure this is your actual test file name

# Directory to Save Models & Scaler
os.makedirs("2Ddata_Transformer", exist_ok=True)

# Label Columns
label_columns = ["Positive_Emotions", "Negative_Emotions", "Self_Esteem", "Meaning_in_Life", "Social_Support"]

# ===== Step 1: Compute Global Max Sequence Length Across All Folds ===== #
global_max_seq_length = 0

for file_name in fold_files:
    with open(file_name, "rb") as f:
        fold_dict = pickle.load(f)

    max_length = max(len(seq) for seq in fold_dict["data"])
    global_max_seq_length = max(global_max_seq_length, max_length)

print(f"\n✅ Global Max Sequence Length: {global_max_seq_length}")

# ===== Step 2: Standardize Features Using Training Data Only ===== #
scaler = StandardScaler()

# Fit scaler on all training folds
all_train_features = []

for file_name in fold_files:
    with open(file_name, "rb") as f:
        fold_dict = pickle.load(f)

    columns = fold_dict["columns"]
    label_indices = [columns.index(col) for col in label_columns]
    feature_indices = [i for i in range(len(columns)) if i not in label_indices and columns[i] != "participant_id"]

    X_raw = [np.array(seq[:, feature_indices], dtype=np.float32) for seq in fold_dict["data"]]
    X_flattened = np.vstack(X_raw)  # Flatten sequences for fitting scaler
    all_train_features.append(X_flattened)

# Fit scaler on the entire training set
scaler.fit(np.vstack(all_train_features))

# Save Scaler
with open("2Ddata_Transformer/scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("✅ StandardScaler saved!")

# ===== Step 3: Preprocess the Test Set ===== #
with open(test_file, "rb") as f:
    test_dict = pickle.load(f)

columns = test_dict["columns"]
label_indices = [columns.index(col) for col in label_columns]
feature_indices = [i for i in range(len(columns)) if i not in label_indices and columns[i] != "participant_id"]

X_test_raw = [np.array(seq[:, feature_indices], dtype=np.float32) for seq in test_dict["data"]]
y_test_raw = [np.array(seq[:, label_indices], dtype=np.float32) for seq in test_dict["data"]]

# Standardize Test Features Using Pre-Saved Scaler
X_test_flat = np.vstack(X_test_raw)
X_test_scaled = scaler.transform(X_test_flat)

# Restore sequence structure
X_test_fixed = []
start = 0
for seq in X_test_raw:
    length = len(seq)
    X_test_fixed.append(X_test_scaled[start:start+length])
    start += length

# Apply Padding
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(
    X_test_fixed, maxlen=global_max_seq_length, dtype="float32", padding="post"
)

# Extract last timestep labels
y_test_final = np.array([seq[-1] for seq in y_test_raw], dtype=np.float32)

print(f"\n✅ Test set preprocessed! Test samples: {len(X_test_padded)}")



✅ Global Max Sequence Length: 74
✅ StandardScaler saved!

✅ Test set preprocessed! Test samples: 52


In [8]:
model.evaluate(X_test_padded, y_test_final, verbose=1)

2/2 [==============================] - 0s 8ms/step - loss: 1.8922 - mae: 1.2810


[1.8922131061553955, 1.280975580215454]

In [10]:
train_samples = 0

for file_name in fold_files:
    with open(file_name, "rb") as f:
        fold_dict = pickle.load(f)
    
    train_samples += len(fold_dict["data"])  # Each sequence is one sample

print(f"✅ Total Training Samples: {train_samples}")


✅ Total Training Samples: 1052


In [11]:
with open("test_2d_fold5.pkl", "rb") as f:
    test_dict = pickle.load(f)

test_samples = len(test_dict["data"])
print(f"✅ Test Samples: {test_samples}")


✅ Test Samples: 52


In [12]:
val_samples = int(train_samples * 0.2 / NUM_FOLDS)  # Per fold
print(f"✅ Validation Samples per Fold: {val_samples}")


✅ Validation Samples per Fold: 42


In [1]:
# wrong?

import numpy as np
import pickle
import tensorflow as tf
import os
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, MultiHeadAttention, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Number of Folds
NUM_FOLDS = 5
fold_files = [f"train_2d_fold{i}.pkl" for i in range(1, 6)]
test_files = [f"test_2d_fold{i}.pkl" for i in range(1, 6)]

# Directory to Save Models & Scaler
os.makedirs("2Ddata_Transformer", exist_ok=True)

# Label Columns
label_columns = ["Positive_Emotions", "Negative_Emotions", "Self_Esteem", "Meaning_in_Life", "Social_Support"]

# ===== Step 1: Compute Global Max Sequence Length Across All Folds ===== #
global_max_seq_length = 0
for file_name in fold_files:
    with open(file_name, "rb") as f:
        fold_dict = pickle.load(f)
    max_length = max(len(seq) for seq in fold_dict["data"])
    global_max_seq_length = max(global_max_seq_length, max_length)

print(f"\n✅ Global Max Sequence Length: {global_max_seq_length}")

# ===== Step 2: Standardize Features Using All Training Data ===== #
scaler = StandardScaler()
all_train_features = []

for file_name in fold_files:
    with open(file_name, "rb") as f:
        fold_dict = pickle.load(f)
    columns = fold_dict["columns"]
    label_indices = [columns.index(col) for col in label_columns]
    feature_indices = [i for i in range(len(columns)) if i not in label_indices and columns[i] != "participant_id"]
    X_raw = [np.array(seq[:, feature_indices], dtype=np.float32) for seq in fold_dict["data"]]
    X_flattened = np.vstack(X_raw)
    all_train_features.append(X_flattened)

scaler.fit(np.vstack(all_train_features))

# Save Scaler
with open("2Ddata_Transformer/scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("✅ StandardScaler saved!")

# ===== Step 3: Preprocess the Test Set ===== #
test_samples = 0
test_data = []
y_test = []

for test_file in test_files:
    with open(test_file, "rb") as f:
        test_dict = pickle.load(f)
    columns = test_dict["columns"]
    label_indices = [columns.index(col) for col in label_columns]
    feature_indices = [i for i in range(len(columns)) if i not in label_indices and columns[i] != "participant_id"]
    X_test_raw = [np.array(seq[:, feature_indices], dtype=np.float32) for seq in test_dict["data"]]
    y_test_raw = [np.array(seq[:, label_indices], dtype=np.float32) for seq in test_dict["data"]]
    X_test_flat = np.vstack(X_test_raw)
    X_test_scaled = scaler.transform(X_test_flat)
    X_test_fixed = []
    start = 0
    for seq in X_test_raw:
        length = len(seq)
        X_test_fixed.append(X_test_scaled[start:start+length])
        start += length
    X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(
        X_test_fixed, maxlen=global_max_seq_length, dtype="float32", padding="post"
    )
    test_data.append(X_test_padded)
    y_test.extend([seq[-1] for seq in y_test_raw])
    test_samples += len(X_test_padded)

y_test_final = np.array(y_test, dtype=np.float32)
print(f"\n✅ Test set preprocessed! Total test samples: {test_samples}")


2025-03-05 20:48:52.826750: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-05 20:48:52.864264: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-05 20:48:52.864288: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-05 20:48:52.864325: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 20:48:52.873111: I tensorflow/core/platform/cpu_feature_g


✅ Global Max Sequence Length: 74
✅ StandardScaler saved!

✅ Test set preprocessed! Total test samples: 263


In [2]:
# ===== Step 4: Define Transformer Model ===== #
def build_transformer_model(input_shape):
    inputs = Input(shape=input_shape)
    attn_output = MultiHeadAttention(num_heads=2, key_dim=32)(inputs, inputs)
    attn_output = Dropout(0.5)(attn_output)
    attn_output = LayerNormalization(epsilon=1e-6)(attn_output + inputs)
    ffn = Dense(64, activation="relu", kernel_regularizer=l2(0.02))(attn_output)
    ffn = Dropout(0.5)(ffn)
    ffn = Dense(32, activation="relu", kernel_regularizer=l2(0.02))(ffn)
    outputs = Dense(5, activation="linear")(ffn[:, -1, :])
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss="mae", metrics=["mae"])
    return model

# ===== Step 5: Train and Evaluate the Model ===== #
model = build_transformer_model((global_max_seq_length, len(feature_indices)))
model.fit(
    np.vstack(test_data), y_test_final, epochs=50, batch_size=8, validation_split=0.2, verbose=1
)

# Save the Model
model.save("2Ddata_Transformer/final_transformer_model.keras")
print("✅ Final model saved!")

Epoch 1/50
27/27 [==============================] - 2s 23ms/step - loss: 6.7350 - mae: 4.0375 - val_loss: 6.5011 - val_mae: 3.8203
Epoch 2/50
27/27 [==============================] - 0s 16ms/step - loss: 6.2891 - mae: 3.6211 - val_loss: 5.9936 - val_mae: 3.3394
Epoch 3/50
27/27 [==============================] - 0s 16ms/step - loss: 5.9733 - mae: 3.3308 - val_loss: 5.5818 - val_mae: 2.9525
Epoch 4/50
27/27 [==============================] - 0s 16ms/step - loss: 5.6353 - mae: 3.0172 - val_loss: 5.1996 - val_mae: 2.5941
Epoch 5/50
27/27 [==============================] - 0s 16ms/step - loss: 5.3304 - mae: 2.7358 - val_loss: 4.8664 - val_mae: 2.2844
Epoch 6/50
27/27 [==============================] - 0s 16ms/step - loss: 5.1555 - mae: 2.5844 - val_loss: 4.6355 - val_mae: 2.0770
Epoch 7/50
27/27 [==============================] - 0s 16ms/step - loss: 4.9844 - mae: 2.4370 - val_loss: 4.4630 - val_mae: 1.9278
Epoch 8/50
27/27 [==============================] - 0s 16ms/step - loss: 4.7633 - m

In [16]:
# Evaluate on Test Set
test_loss, test_mae = model.evaluate(np.vstack(test_data), y_test_final, verbose=1)
print(f"\n✅ Final Test Loss: {test_loss:.4f}, Test MAE: {test_mae:.4f}")

9/9 [==============================] - 0s 5ms/step - loss: 2.8670 - mae: 1.2531

✅ Final Test Loss: 2.8670, Test MAE: 1.2531
